# 1. Data Analysis

Librerias relevantes para el análisis

In [24]:
library(rio)
library(ggplot2)
library(caTools)
library(DescTools)
library(dplyr)
library(Hmisc)
library(psych)

In [25]:
url="https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"
wage<-read.csv(url)

In [26]:
names(wage)

[1] "rownames" "wage"     "lwage"    "sex"      "shs"      "hsg"     
 [7] "scl"      "clg"      "ad"       "mw"       "so"       "we"      
[13] "ne"       "exp1"     "exp2"     "exp3"     "exp4"     "occ"     
[19] "occ2"     "ind"      "ind2"

2. Reporte de NAs


In [27]:
colSums(is.na(wage))

rownames     wage    lwage      sex      shs      hsg      scl      clg 
       0        0        0        0        0        0        0        0 
      ad       mw       so       we       ne     exp1     exp2     exp3 
       0        0        0        0        0        0        0        0 
    exp4      occ     occ2      ind     ind2 
       0        0        0        0        0

No existen datos perdidos en la base de datos

### 3.  Reporte de estadísticas descriptivas

In [28]:
describe(wage$wage)

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
X1,1,5150,23.41041,21.00302,19.23077,20.53559,10.26415,3.021978,528.8457,525.8237,10.76153,210.9187,0.2926699


**Interpretación** : se puede extraer información relevante si nos enfocamos en la variable del salario del individuo. Se puede observar que el salario promedio de la muestra es de 23.41 dólares.No obstante, la desviación estándar es muy alta(21), lo cual quiere decir que existe una alta dispersión del salario con respecto al promedio. Por ejemplo, existe un individuo cuyo salario es de 528.85 dólares y otro cuyo salario es de 3.02 dólares. En aras de obtener información más estable sobre el salario se recurre a la mediana, la cual nos dice que el 50% de la muestra posee un salario entre los 3.02 y 19.23 dólares. Por último, la muestra presenta una asimetría positiva, lo cual implica que existe un mayor número de trabajadores con un salario menor al promedio.

4. Obtener el número de mujeres con un grado académico alto que poseen un salario del 25% más rico de la muestra

4.1 Identificación de cuantiles

In [29]:
summary(wage$wage)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  3.022  13.461  19.231  23.410  27.778 528.846 

4.2 Creamos nueva variable

In [30]:
wage$wage_ordinal=cut(wage$wage, breaks = c(0,13.461,19.231,27.778,528.846),
                      include.lowest = T, ordered_result = T,
                      labels=c("Muy bajo", "Bajo", "Alto","Muy alto"))
table(wage$wage_ordinal)


Muy bajo     Bajo     Alto Muy alto 
    1269     1462     1132     1287 

4.3 Creamos dos subsets que solo contengan a uno de los sexos

In [31]:
wage_only_women=wage%>%filter(sex == 1)
wage_only_men= wage%>%filter(sex == 0)
wage_only_women$case<- c(1)
wage_only_men$case<- c(1)

4.4 Agrupamos

In [32]:
wage_only_women%>%group_by(clg,wage_ordinal)%>%summarise(Casos=sum(case))

`summarise()` has grouped output by 'clg'. You can override using the `.groups`
argument.


clg,wage_ordinal,Casos
<dbl>,<ord>,<dbl>
0,Muy bajo,484
0,Bajo,434
0,Alto,281
0,Muy alto,295
1,Muy bajo,107
1,Bajo,227
1,Alto,224
1,Muy alto,237



Existen 237 mujeres con un college dregee que poseen un salario del 25% más rico de la muestra.

In [33]:
wage_only_women%>%group_by(ad,wage_ordinal)%>%summarise(Casos=sum(case))

`summarise()` has grouped output by 'ad'. You can override using the `.groups`
argument.


ad,wage_ordinal,Casos
<dbl>,<ord>,<dbl>
0,Muy bajo,558
0,Bajo,581
0,Alto,392
0,Muy alto,357
1,Muy bajo,33
1,Bajo,80
1,Alto,113
1,Muy alto,175


Existen 175 mujeres con un advanced degree que poseen un salario del 25% más rico de la muestra. En ambos dataframes, las mujeres que no poseen alguno de estos grados académicos tienden a ser más propensas a tener un salario del 25% menos rico de la muestra, en comparación a las mujeres que sí poseen un mayor nivel académico. Ello nos lleva a inferir que probablemente el nivel educativo puede influir en la obtención de mejores salarios.

5. Obtener el número de hombres con un grado académico medio-bajo que poseen un salario perteneciente al 25% más rico de la muestra

In [34]:
wage_only_men%>%group_by(hsg, wage_ordinal)%>%summarise(Casos=sum(case))

`summarise()` has grouped output by 'hsg'. You can override using the `.groups`
argument.


hsg,wage_ordinal,Casos
<dbl>,<ord>,<dbl>
0,Muy bajo,369
0,Bajo,525
0,Alto,477
0,Muy alto,648
1,Muy bajo,309
1,Bajo,276
1,Alto,150
1,Muy alto,107


Existen 107 trabajadores hombres con secundaria completa que poseen un salario perteneciente al 25% más rico de la muestra.

In [35]:
wage_only_men%>%group_by(shs, wage_ordinal)%>%summarise(Casos=sum(case))

`summarise()` has grouped output by 'shs'. You can override using the `.groups`
argument.


shs,wage_ordinal,Casos
<dbl>,<ord>,<dbl>
0,Muy bajo,634
0,Bajo,775
0,Alto,614
0,Muy alto,747
1,Muy bajo,44
1,Bajo,26
1,Alto,13
1,Muy alto,8


Existen solo 8 trabajadores hombres con secundaria incompleta que tienen un salario perteneciente al 25% más rico de la muestra. Para ambos casos, los hombres que no poseen alguno de estos grados educativos son más propensos a poseer un salario pertenenciente al 25% más pobre. Si comparamos estos resultados con el anterior grupo, podemos observar que hay más mujeres con un salario altísimo en comparación con los hombres, pero ello puede deberse al mayor nivel educativo que poseen.

6. Creación de dos bases de datos

In [36]:
Y=wage%>%select(3)

In [37]:
X=wage%>%select(!(2:3))